In [1]:
# On importe la librairie bigml
from bigml.api import BigML

In [2]:
# On se connecte à bigml
api = BigML(project='project/5d94a32e42129f2e16000232')

In [3]:
# On récupère un ensemble
ensemble = api.get_ensemble('ensemble/5dc2f3f07811dd7f2d012312')

In [12]:
# On crée une prédiction
prediction = api.create_prediction(ensemble, {
        "RevolvingUtilizationOfUnsecuredLines": 0.01703559,
        "NumberOfDependents": 1,
        "DebtRatio": 0,
        "age": 42,
        "NumberOfOpenCreditLinesAndLoans": 6,
        "NumberRealEstateLoansOrLines": 1,
        "NumberOfTime30-59DaysPastDueNotWorse": 1,
        "NumberOfTime60-89DaysPastDueNotWorse": 0,
        "NumberOfTimes90DaysLate": 0
    },
    {
        "name" : "Exercice Louis"
    }
)

In [13]:
# On affiche la prédiction
api.pprint(prediction)

SeriousDlqin2yrs for {'DebtRatio': 0, 'NumberOfDependents': 1, 'NumberOfOpenCreditLinesAndLoans': 6, 'NumberOfTime30-59DaysPastDueNotWorse': 1, 'NumberOfTime60-89DaysPastDueNotWorse': 0, 'NumberOfTimes90DaysLate': 0, 'NumberRealEstateLoansOrLines': 1, 'RevolvingUtilizationOfUnsecuredLines': 0.01703559, 'age': 42} is 0
